In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

data_dir = "/content/drive/MyDrive/vietnamese_money_dataset"  # đổi nếu khác

os.listdir(data_dir)


['500000 VND', '20000 VND', '200000 VND', '50000 VND', '100000 VND']

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_gen.class_indices)
print("Các lớp:", train_gen.class_indices)


Found 261 images belonging to 5 classes.
Found 64 images belonging to 5 classes.
Các lớp: {'100000 VND': 0, '20000 VND': 1, '200000 VND': 2, '50000 VND': 3, '500000 VND': 4}


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,483,397 (32.36 MB)

 Trainable params: 8,483,397 (32.36 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
    train_gen,
    epochs=25,
    validation_data=val_gen,
    verbose=1
)

model.save("/content/drive/MyDrive/money_cnn_model.keras")


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 201s 22s/step - accuracy: 0.2248 - loss: 2.1423 - val_accuracy: 0.2656 - val_loss: 1.5780
Epoch 2/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 491ms/step - accuracy: 0.3191 - loss: 1.5595 - val_accuracy: 0.3594 - val_loss: 1.4140
Epoch 3/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 577ms/step - accuracy: 0.3924 - loss: 1.3870 - val_accuracy: 0.5938 - val_loss: 1.1754
Epoch 4/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 481ms/step - accuracy: 0.5817 - loss: 1.1115 - val_accuracy: 0.7188 - val_loss: 0.9068
Epoch 5/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 498ms/step - accuracy: 0.6996 - loss: 0.8518 - val_accuracy: 0.7188 - val_loss: 0.7612
Epoch 6/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 600ms/step - accuracy: 0.7771 - loss: 0.7280 - val_accuracy: 0.4688 - val_loss: 1.6990
Epoch 7/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 494ms/step - accuracy: 0.5840 - loss: 1.1480 - val_accuracy: 0.6875 - val_loss: 1.0224
Epoch 8/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 508ms/step - accuracy: 0.7622 - loss: 0.8336 - val_accuracy: 0.7656 - val_loss:

In [6]:
money_info = {
    "20000 VND": "Tờ 20,000 VNĐ: màu xanh da trời, in hình Chủ tịch Hồ Chí Minh ở mặt trước và chùa Cầu Hội An ở mặt sau.",
    "50000 VND": "Tờ 50,000 VNĐ: màu xanh lá - nâu, in hình Chủ tịch Hồ Chí Minh ở mặt trước và cảnh Phong Nha - Kẻ Bàng ở mặt sau.",
    "100000 VND": "Tờ 100,000 VNĐ: màu xanh lá cây, in hình Chủ tịch Hồ Chí Minh và chùa Văn Miếu Quốc Tử Giám.",
    "200000 VND": "Tờ 200,000 VNĐ: màu đỏ cam, in hình Chủ tịch Hồ Chí Minh và cầu Hiền Lương - sông Bến Hải.",
    "500000 VND": "Tờ 500,000 VNĐ: màu xanh dương, tờ có mệnh giá lớn nhất, in hình Chủ tịch Hồ Chí Minh và quê hương của Bác ở Nghệ An."
}


In [7]:
import numpy as np
from tensorflow.keras.preprocessing import image

labels = {v: k for k, v in train_gen.class_indices.items()}

def predict_money(img):
    img = img.resize(img_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) / 255.0
    preds = model.predict(x)
    class_index = np.argmax(preds[0])
    class_name = labels[class_index]
    return f"{class_name}\n\n{money_info[class_name]}"


In [8]:
from tensorflow.keras.models import load_model

# Load model đã lưu trước đó
model = load_model("/content/drive/MyDrive/money_cnn_model.keras")

print("✅ Model đã load thành công, sẵn sàng dự đoán!")


✅ Model đã load thành công, sẵn sàng dự đoán!


In [9]:
import gradio as gr

def predict_money_ui(img):
    result = predict_money(img)
    lines = result.split("\n\n")
    title = lines[0]
    desc = lines[1] if len(lines) > 1 else ""

    html_result = f"""
    <div style="font-size:26px; font-weight:bold; color:#2c3e50; margin-bottom:15px;">
        {title}
    </div>
    <div style="font-size:18px; line-height:1.6; text-align:justify; color:#34495e;">
        {desc}
    </div>
    """
    return img, html_result

demo = gr.Interface(
    fn=predict_money_ui,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Image(label="Ảnh tiền upload", type="pil"),
        gr.HTML(label="Kết quả nhận diện")
    ],
    title="💵 Nhận diện tiền Việt Nam",
    description="Upload ảnh tờ tiền Việt Nam để hệ thống nhận diện mệnh giá và hiển thị thông tin chi tiết."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed5ec361079f2f1866.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
